## NDBC module

https://supermanzer.github.io/NDBC/html/NDBC.html


In [2]:
from NDBC.NDBC import DataBuoy
import folium
import re
import matplotlib.pyplot as plt
from windrose import WindroseAxes, plot_windrose
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [12]:
DB = DataBuoy()
DB.set_station_id("42050")
# DB.set_station_id('42091')

DB.get_station_metadata()
DB.station_info

{'lat': '28.842 N',
 'lon': '94.242 W',
 'Site elevation': 'sea level',
 'Sea temp depth': '2 m below MLLW',
 'Water depth': '24 m'}

In [9]:
DB.get_data(months=range(10, 12), years=range(1996, 1997), datetime_index=True)

Oct not available.
Nov not available.
Please review available station data:
 https://www.ndbc.noaa.gov/station_history.php?station=42035


In [11]:
df = DB.data["stdmet"]["data"]
df

,WD,WSPD,GST,WVHT,DPD,APD,MWD,BAR,ATMP,WTMP,DEWP,VIS
1996-01-01 00:00:00,194.0,5.8,6.7,0.89,6.67,4.54,NaN,1002.4,14.8,14.3,NaN,NaN
1996-01-01 01:00:00,188.0,5.4,6.3,0.91,6.25,4.67,NaN,1002.8,15.2,14.3,NaN,NaN
1996-01-01 02:00:00,179.0,3.3,3.8,0.80,6.67,4.65,NaN,1002.7,15.2,14.2,NaN,NaN
1996-01-01 03:00:00,159.0,4.4,5.0,0.82,7.14,4.93,NaN,1002.6,15.0,14.2,NaN,NaN
1996-01-01 04:00:00,129.0,3.3,3.7,0.79,6.25,4.73,NaN,1002.4,15.2,14.2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1996-12-31 19:00:00,NaN,NaN,NaN,0.63,3.70,3.67,NaN,1018.7,16.2,15.7,NaN,NaN
1996-12-31 20:00:00,NaN,NaN,NaN,0.61,3.57,3.69,NaN,1018.2,16.4,15.7,NaN,NaN
1996-12-31 21:00:00,NaN,NaN,NaN,0.56,3.85,3.70,NaN,1018.3,16.6,15.8,NaN,NaN
1996-12-31 22:00:00,NaN,NaN,NaN,0.53,3.70,3.97,NaN,1018.6,16.8,15.8,NaN,NaN


In [8]:
stations = DB.station_search(distance=400)

In [16]:
DB = DataBuoy()
DB.set_station_id("42035")
DB.get_station_metadata()
latitude = float(re.findall(r"[-+]?\d*\.\d+|\d+", DB.station_info["lat"])[0])
longitude = -float(re.findall(r"[-+]?\d*\.\d+|\d+", DB.station_info["lon"])[0])
location_name = "42035"

map = folium.Map(location=[latitude, longitude], zoom_start=6)
folium.Marker([latitude, longitude], popup=location_name).add_to(map)

# Add markers for other stations returned by station_search
# for station in DB.station_search(distance=400):
# for station in stations:
stations = [42035, 42043, 42050, 42091]
for station in stations:
    DB_ = DataBuoy()
    DB_.set_station_id(station)
    DB_.get_station_metadata()
    latitude = float(re.findall(r"[-+]?\d*\.\d+|\d+", DB_.station_info["lat"])[0])
    longitude = -float(re.findall(r"[-+]?\d*\.\d+|\d+", DB_.station_info["lon"])[0])
    location_name = station
    folium.Marker([latitude, longitude], popup=location_name).add_to(map)

map

In [24]:
import pandas as pd


def process_buoy_data(station_id):
    DB = DataBuoy()
    DB.set_station_id(station_id)

    final_df = pd.DataFrame()

    for year in range(2020, 2021):
        for month in range(1, 12):
            try:
                DB.get_data(months=[month], years=[year], datetime_index=True)
                df = DB.data["stdmet"]["data"]

                column_name_mapping = {
                    "WDIR": "wind_direction",
                    "WSPD": "wind_speed",
                    "GST": "wind_gust",
                    "WVHT": "wave_height",
                    "DPD": "dominant_wave_period",
                    "APD": "average_wave_period",
                    "MWD": "mean_wave_direction",
                    "PRES": "pressure",
                    "ATMP": "air_temperature",
                    "WTMP": "water_temperature",
                    "DEWP": "dewpoint",
                    "VIS": "visibility",
                    "TIDE": "3hr_pressure_tendency",
                }

                df.rename(columns=column_name_mapping, inplace=True)

                df["Year"] = df.index.year
                df["Month"] = df.index.month

                final_df = pd.concat([final_df, df])
            except Exception as e:
                print(f"Error processing data for {year}-{month}: {e}")

    excel_filename = f"Offshore_Wave_{station_id}_data.xlsx"
    final_df.to_excel(excel_filename, index=True)

    return final_df

In [27]:
pwd

'/Users/abolfazl/Desktop/ongoingProjects/CedarLakes/Data/Offshore Wave Data'

In [26]:
process_buoy_data(42035)

/opt/anaconda3/lib/python3.11/site-packages/NDBC/NDBC.py:364: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv(url, sep=r"\s+")
Jan not available.
Please review available station data:
 https://www.ndbc.noaa.gov/station_history.php?station=42035
/opt/anaconda3/lib/python3.11/site-packages/NDBC/NDBC.py:364: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv(url, sep=r"\s+")
Feb not available.
Please review available station data:
 https://www.ndbc.noaa.gov/station_history.php?station=42035


Error processing data for 2020-2: Reindexing only valid with uniquely valued Index objects


/opt/anaconda3/lib/python3.11/site-packages/NDBC/NDBC.py:364: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv(url, sep=r"\s+")


Error processing data for 2020-3: Reindexing only valid with uniquely valued Index objects


/opt/anaconda3/lib/python3.11/site-packages/NDBC/NDBC.py:364: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv(url, sep=r"\s+")


...

/opt/anaconda3/lib/python3.11/site-packages/NDBC/NDBC.py:364: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv(url, sep=r"\s+")


Error processing data for 2020-10: Reindexing only valid with uniquely valued Index objects


/opt/anaconda3/lib/python3.11/site-packages/NDBC/NDBC.py:364: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv(url, sep=r"\s+")


Error processing data for 2020-11: Reindexing only valid with uniquely valued Index objects


,wind_direction,wind_speed,wind_gust,wave_height,dominant_wave_period,average_wave_period,mean_wave_direction,pressure,air_temperature,water_temperature,dewpoint,visibility,3hr_pressure_tendency,Year,Month
2020-01-01 00:00:00,NaN,0.9,1.4,NaN,NaN,NaN,NaN,1022.7,NaN,NaN,NaN,NaN,NaN,2020,1
2020-01-01 00:10:00,NaN,1.1,1.7,NaN,NaN,NaN,NaN,1022.6,NaN,NaN,NaN,NaN,NaN,2020,1
2020-01-01 00:20:00,NaN,1.0,1.7,NaN,NaN,NaN,NaN,1022.8,NaN,NaN,NaN,NaN,NaN,2020,1
2020-01-01 00:30:00,NaN,1.2,1.8,NaN,NaN,NaN,NaN,1022.7,NaN,NaN,NaN,NaN,NaN,2020,1
2020-01-01 00:40:00,NaN,1.2,1.8,0.21,NaN,3.43,NaN,1022.6,NaN,NaN,NaN,NaN,NaN,2020,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 23:10:00,239.0,12.5,15.4,NaN,NaN,NaN,NaN,1004.5,13.0,15.9,12.4,NaN,NaN,2020,12
2020-12-31 23:20:00,242.0,12.5,15.6,NaN,NaN,NaN,NaN,1004.2,12.9,15.9,12.2,NaN,NaN,2020,12
2020-12-31 23:30:00,246.0,11.9,15.5,NaN,NaN,NaN,NaN,1004.6,12.8,15.9,12.0,NaN,NaN,2020,12
2020-12-31 23:40:00,248.0,12.5,15.3,NaN,NaN,NaN,NaN,1004.5,12.5,15.9,11.6,NaN,NaN,2020,12


In [19]:
for station in station:
    process_buoy_data(station)

TypeError: 'int' object is not iterable

In [ ]:
col_units = {
    "wind_direction": "degrees",
    "wind_speed": "meters/second",
    "wind_gust": "meters/second",
    "wave_height": "meters",
    "dominant_wave_period": "seconds",
    "average_wave_period": "seconds",
    "dominant_wave_direction": "degrees",
    "pressure": "hPa",
    "air_temperature": "degC",
    "water_temperature": "degC",
    "dewpoint": "degC",
    "visibility": "nautical_mile",
    "3hr_pressure_tendency": "hPa",
    "water_level_above_mean": "feet",
    "time": None,
}

In [ ]:
def remove_rows_with_missing_values(df, column1, column2):
    return df.dropna(subset=[column1, column2])


df_ = remove_rows_with_missing_values(df, "wind_direction", "wind_speed")

In [ ]:
def plot_windrose_subplots(data, *, direction, var, color=None, **kwargs):
    """wrapper function to create subplots per axis"""
    ax = plt.gca()
    ax = WindroseAxes.from_ax(ax=ax)
    plot_windrose(direction_or_df=data[direction], var=data[var], ax=ax, **kwargs)

In [ ]:
g = sns.FacetGrid(
    data=df_,
    # the column name for each level a subplot should be created
    col="Year",
    # place a maximum of 3 plots per row
    col_wrap=3,
    subplot_kws={"projection": "windrose"},
    sharex=False,
    sharey=False,
    despine=False,
    height=3.5,
)

g.map_dataframe(
    plot_windrose_subplots,
    direction="wind_direction",
    var="wind_speed",
    normed=True,
    # manually set bins, so they match for each subplot
    bins=(0.1, 3, 6, 9, 12, 15),
    calm_limit=0.1,
    kind="bar",
)

y_ticks = range(0, 17, 4)
for ax in g.axes:
    ax.set_legend(
        title="$m \cdot s^{-1}$", bbox_to_anchor=(1.15, -0.1), loc="lower right"
    )
    ax.set_rgrids(y_ticks, y_ticks)

# adjust the spacing between the subplots to have sufficient space between plots
plt.subplots_adjust(wspace=-0.2)

In [ ]:
df_ = remove_rows_with_missing_values(df, "mean_wave_direction", "wave_height")

g = sns.FacetGrid(
    data=df_,
    # the column name for each level a subplot should be created
    col="Year",
    # place a maximum of 3 plots per row
    col_wrap=3,
    subplot_kws={"projection": "windrose"},
    sharex=False,
    sharey=False,
    despine=False,
    height=3.5,
)

g.map_dataframe(
    plot_windrose_subplots,
    direction="mean_wave_direction",
    var="wave_height",
    normed=True,
    # manually set bins, so they match for each subplot
    bins=(0.1, 1, 2, 3, 4, 5),
    calm_limit=0.1,
    kind="bar",
)

y_ticks = range(0, 17, 4)
for ax in g.axes:
    ax.set_legend(title="$m$", bbox_to_anchor=(1.15, -0.1), loc="lower right")
    ax.set_rgrids(y_ticks, y_ticks)

# adjust the spacing between the subplots to have sufficient space between plots
plt.subplots_adjust(wspace=-0.2)

In [ ]:
df_

In [ ]:
sns.displot(
    data=df_,
    x="wind_speed",
    col="Year",
    hue="Month",
    kind="kde",
    col_wrap=3,
    height=3.5,
    facet_kws={"sharex": False, "sharey": False},
    common_norm=False,  # Normalize the KDE for each hue category separately
)

In [ ]:
sns.displot(
    data=df_,
    x="wind_speed",
    col="Year",
    hue="Month",
    kind="ecdf",
    col_wrap=3,
    height=3.5,
    facet_kws={"sharex": False, "sharey": False},
)

In [ ]:
df_42002.index[0], df_42002.index[-1]

In [ ]:
df_42019.index[0], df_42019.index[-1]

In [ ]:
df_42020.index[0], df_42020.index[-1]

In [ ]:
df_42002